### Fetching Ugandan news data

**Import the `BeautifulSoup` library for web scraping and `requests` for making web requests**

In [10]:
import requests
from bs4 import BeautifulSoup
import datetime

**Setting up `pymongo`, a package used to connect with `MongoDB`, where the news data will be stored**

In [11]:
import pymongo
db_client = pymongo.MongoClient()
news_db = db_client["ug_news"]
col = news_db["news"]

**Fetch data from the `New Vision` online coverpage**

In [12]:
url = "https://www.newvision.co.ug"
news_href = "https://www.newvision.co.ug/new_vision/news/"

news_request = requests.get(url)
coverpage = news_request.content

**Pick out the anchor tags to get the links to the actual news articles**

In [13]:
soup1 = BeautifulSoup(coverpage, 'html5lib')
article_links = soup1.find_all('a')

**Follow each link and fetch the article content**

In [17]:
text = []

for link in article_links:
    if news_href in link['href']:
        article = requests.get(link['href'])
        article_content = article.content
        soup2 = BeautifulSoup(article_content, 'html5lib')
        body = soup2.find('div', class_='article-content')
        soup3 = BeautifulSoup(article_content, 'html5lib')
        paragraphs = soup3.find_all('p')
        for p in paragraphs:
            text.append(p.get_text())
            
final_text = " ".join(text)
timestamp = datetime.datetime.now()
news_entry = {timestamp.strftime("%m/%d/%Y, %H:%M:%S"): final_text}
print(news_entry)

{'05/20/2020, 08:40:12': "\n\t\t\t\t\t\t\tBy Miriam Namutebi\n\t\t\t\t\t\t \n\t\t\t\t\t\t\tAdded 19th May 2020 08:53 PM\n\t\t\t\t\t\t                                     \n\t\t\t\tIn the fight against the COVID-19 pandemic, the Parliament has led by example. The legislators have put government measures meant to curb the virus into action. The Speaker Rebecca Kadaga and her commission went to the drawing board and came up with a way of implementing the measures.\n\t\t\t Copyright © 2019 New Vision. All Rights Reserved. \n\t\t\t\t\t\tKindly provide your details to receive the Newvision Newsletter\n\t\t\t\t\t \n\t\t\t\t\t\tBy Julius  Luwemba\n\t\t\t\t\t \n\t\t\t\t\t\tAdded 19th May 2020 08:48 PM\n\t\t\t\t\t \n\t\t\t\tWhat used to be a compound, is now an extension of Lake Victoria and the nuns have to use a boat if they're to access any corner of the compound, or crossing from one dormitory to another.\n\t\t\t \n\t\t\t\t\t\t The building which was opened by Arcgbishop Cyprian Kizito Lwang

**Store text in `mongodb` database**

In [19]:
col.insert_one(news_entry)